Let's train the guy

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm as tqdm
tf.keras.backend.set_floatx('float32')
from collections import deque
from datetime import datetime
import random
import matplotlib
from environment import Environment
from plots import just_plot
from misc import *
from nets import *
from buffer import ReplayBuffer


amplitude=0.4
tau = 0.1
lr_critic = 0.005
lr_actor=10**-5
noise_displacement = .25
ep_guess=0.01
dolinar_layers=1
number_phases=2
buffer_size = 10**7


env = Environment(amplitude=amplitude, dolinar_layers = dolinar_layers, number_phases=number_phases)
buffer = ReplayBuffer(buffer_size=buffer_size)

critic = Critic(nature="primary",valreg=0.0, dolinar_layers = dolinar_layers, number_phases=number_phases)
critic_target = Critic(nature="target", valreg=0.0,dolinar_layers = dolinar_layers, number_phases=number_phases, tau = tau)
actor = Actor(nature="primary",valreg=0.0, dolinar_layers = dolinar_layers)
actor_target = Actor(nature="target",valreg=0.0, dolinar_layers = dolinar_layers)

optimizer_critic = tf.keras.optimizers.Adam(lr=lr_critic)
optimizer_actor = tf.keras.optimizers.SGD(lr=lr_actor)

policy_evaluator = PolicyEvaluator(amplitude = amplitude, dolinar_layers=dolinar_layers, number_phases = number_phases)

expes = np.load("buffers/1L-stoch.npy")

for k in tqdm(expes):
    buffer.add(tuple(k))


100%|██████████| 5000/5000 [00:00<00:00, 501855.08it/s]


In [2]:

loss_ev = []
history_betas = []
total_episodes = 10**3
batch_size = 32.           

context_outcome_actor = np.reshape(np.array([actor.pad_value]),(1,1,1)).astype(np.float32)
history_betas.append(np.squeeze(actor(context_outcome_actor)[0]))
   

In [3]:
history_betas

[array(0.02131591, dtype=float32)]

In [4]:
varactor = actor.trainable_variables

In [6]:
experiences = buffer.sample(32).astype(np.float32)

dq_da = critic_grad_tf(critic, experiences)


In [ ]:
actor.reset_states_workaround(new_batch_size=len(experiences))
actor.reset_states_workaround(new_batch_size=1)
actor(context_outcome_actor)

In [42]:
@tf.function
def gradeando11(actor, experiences):
    with tf.GradientTape() as tape:
        tape.watch(actor.trainable_variables)
        finns = [tf.ones((experiences.shape[0], 1,1))*actor.pad_value]
        unstacked_exp = tf.unstack(experiences, axis=1)
        for index in range(1,2*actor.dolinar_layers-2,2):
            finns.append(tf.reshape(unstacked_exp[index], (experiences.shape[0], 1,1)))
        final_preds = tf.concat(finns, axis=1)
        final_predss = actor(final_preds)
    da_dtheta1 = tape.jacobian(final_predss, actor.trainable_variables)
    return da_dtheta1

In [43]:
actor.reset_states_workaround(new_batch_size=experiences.shape[0])

gradeando11(actor, experiences)

ERROR:tensorflow:Got error while pfor was converting op name: "loop_body/StatefulPartitionedCall"
op: "StatefulPartitionedCall"
input: "loop_body/zeros"
input: "gradient_tape/actor/dense_8/Tensordot/Reshape_1"
input: "loop_body/zeros_1"
input: "loop_body/zeros_2"
input: "loop_body/zeros_3"
input: "actor/lstm_2/StatefulPartitionedCall:5"
input: "actor/lstm_2/StatefulPartitionedCall:6"
input: "actor/lstm_2/StatefulPartitionedCall:7"
input: "actor/lstm_2/StatefulPartitionedCall:8"
input: "actor/lstm_2/StatefulPartitionedCall:9"
input: "actor/lstm_2/StatefulPartitionedCall:10"
input: "actor/lstm_2/StatefulPartitionedCall:11"
input: "actor/lstm_2/StatefulPartitionedCall:12"
input: "actor/lstm_2/StatefulPartitionedCall:13"
input: "actor/lstm_2/StatefulPartitionedCall:14"
input: "actor/lstm_2/StatefulPartitionedCall:15"
input: "actor/lstm_2/StatefulPartitionedCall:16"
input: "actor/lstm_2/StatefulPartitionedCall:17"
input: "actor/lstm_2/StatefulPartitionedCall:18"
input: "actor/lstm_2/Statefu

ERROR:tensorflow:name: "loop_body/StatefulPartitionedCall"
op: "StatefulPartitionedCall"
input: "loop_body/zeros"
input: "gradient_tape/actor/dense_8/Tensordot/Reshape_1"
input: "loop_body/zeros_1"
input: "loop_body/zeros_2"
input: "loop_body/zeros_3"
input: "actor/lstm_2/StatefulPartitionedCall:5"
input: "actor/lstm_2/StatefulPartitionedCall:6"
input: "actor/lstm_2/StatefulPartitionedCall:7"
input: "actor/lstm_2/StatefulPartitionedCall:8"
input: "actor/lstm_2/StatefulPartitionedCall:9"
input: "actor/lstm_2/StatefulPartitionedCall:10"
input: "actor/lstm_2/StatefulPartitionedCall:11"
input: "actor/lstm_2/StatefulPartitionedCall:12"
input: "actor/lstm_2/StatefulPartitionedCall:13"
input: "actor/lstm_2/StatefulPartitionedCall:14"
input: "actor/lstm_2/StatefulPartitionedCall:15"
input: "actor/lstm_2/StatefulPartitionedCall:16"
input: "actor/lstm_2/StatefulPartitionedCall:17"
input: "actor/lstm_2/StatefulPartitionedCall:18"
input: "actor/lstm_2/StatefulPartitionedCall:19"
input: "actor/lstm

StagingError: in user code:

    <ipython-input-34-48ecdc115572>:11 gradeando11  *
        da_dtheta1 = tape.jacobian(final_predss, actor.trainable_variables)
    /home/cooper-cooper/.local/lib/python3.6/site-packages/tensorflow/python/ops/parallel_for/pfor.py:3600 f  *
        [converter._convert_helper(x).t for x in func._func_graph_outputs])
    /home/cooper-cooper/.local/lib/python3.6/site-packages/tensorflow/python/ops/parallel_for/pfor.py:1457 _convert_helper  **
        if flags.FLAGS.op_conversion_fallback_to_while_loop:
    /home/cooper-cooper/.local/lib/python3.6/site-packages/tensorflow/python/platform/flags.py:85 __getattr__
        wrapped(_sys.argv)
    /home/cooper-cooper/.local/lib/python3.6/site-packages/absl/flags/_flagvalues.py:633 __call__
        name, value, suggestions=suggestions)

    UnrecognizedFlagError: Unknown command line flag 'f'
